In [57]:
#from google.colab import drive
#drive.mount("/content/drive")

In [58]:
!pip install rdflib
!pip install pandas
from rdflib import *
from hashlib import sha256

# Create Source Graph

In [59]:
schema = Graph()
schema.bind('tb', 'http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/tbox#')
schema.bind('ab', 'http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/abox#')
tbox = Namespace('http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/tbox#')
abox = Namespace('http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/abox#')
dcat = Namespace('https://www.w3.org/ns/dcat#')
dcterms = Namespace('http://purl.org/dc/terms/')
dqv = Namespace('http://www.w3.org/ns/dqv#')

## Load Data

In [60]:
import pandas as pd

In [61]:
data_path = '/home/acraf/psr/tfm/Fdatavalidation/DataProduct/Data/Explotation/UPENN-GBM_clinical_info_v2.1.csv'
df = pd.read_csv(data_path)
df.info()

## Data Product Profile

### OWNER

In [62]:
metadata_path = "/home/acraf/psr/tfm/Fdatavalidation/DataProduct/Metadata/explotaiton.csv"

In [63]:
meta_df = pd.read_csv(metadata_path)
meta_dp = meta_df.loc[meta_df['name'] == "UPENN-GBM_clinical_info_v2"]
owner = meta_dp.owner.values[0]


### NAME

In [64]:
dataset_name = meta_dp.name.values[0]
dataset_name

add triples

In [65]:
# Name
schema.add((abox[dataset_name], RDF.type, dcat.Dataset))
#identifier
identifier = sha256(dataset_name.encode('utf-8')).hexdigest()
schema.add((abox[dataset_name],dcterms.identifier, Literal(identifier)))
#owner
schema.add((abox[dataset_name], tbox.owner, Literal(owner)))




### ATTRIBUTES


In [66]:
for column in df.columns:
    # Create RDF triple for Attribute
    schema.add((abox[column], RDF.type, tbox.Attribute))
    # Create RDF triple for Attribute
    schema.add((abox[dataset_name], tbox.hasAttribute, abox[column]))
    # rdf for the dataproperty
    schema.add((abox[column], tbox.attribute, Literal(column)))




## DatasetTypeTemplate

### Format

In [67]:
format = meta_dp.format.values[0]
format

In [68]:
# triple
schema.add((abox.Tabular, RDF.type, tbox.DatasetTypeTemplate))

# format
schema.add((abox.Tabular, dcterms['format'], Literal(format)))  # Correct usage of the namespace

# addDTT
schema.add((abox[dataset_name], tbox.hasDTT, abox.Tabular))


### TechnologyAspects

In [69]:
#triple
schema.add((abox.TA1, RDF.type, tbox.TechnologyAspects))

#  addTA
schema.add((abox[dataset_name], tbox.hasTA, abox.TA1))

#TYPE
schema.add((abox.TA1, tbox.typeAcces, abox.Static))
schema.add((abox.Static, RDF.type, tbox.Acces))

# PATH
schema.add((abox.Static, tbox.path, Literal(data_path)))


# SAVE SOURCE_GRAPH

In [70]:
rdf_format = "turtle"
schema.serialize(destination="./schema_metadata.ttl", format=rdf_format)